# Тестовое задание на позицию Junior Analyst
## Часть 2. Техническое задание
В файле «data.xlsx» представлены данные по поступлению денежных средств от клиентов компании N за покупку ее услуг.

### Описание данных
- `client_id` - ID клиента;
- `sum` – сумма денежных средств;
- `status` – статус оплаты;
- `sale` – менеджер, заключивший сделку;
- `new/current` – статус сделки;
- `document` – наличие оригинала подписанного договора с клиентом;
- `receiving_date` – дата получения оригинала договора

### Получение и предобработка данных

In [1]:
# необходимые библиотеки
import pandas as pd
import numpy as np

In [2]:
# данные
data = pd.read_excel('data.xlsx')
data.head()

,client_id,sum,status,sale,new/current,Unnamed: 5,document,receiving_date
0,NaN,NaN,Май 2021,NaN,NaN,NaN,NaN,NaN
1,6.0,11693.7,ВНУТРЕННИЙ,-,-,NaN,-,-
2,14.0,5452.7,ОПЛАЧЕНО,Петрова,новая,NaN,оригинал,2021-07-26 00:00:00
3,15.0,13991.7,ОПЛАЧЕНО,Иванов,текущая,NaN,оригинал,2021-06-08 00:00:00
4,16.0,719.7,ОПЛАЧЕНО,Иванов,текущая,NaN,оригинал,2021-06-08 00:00:00


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   client_id       724 non-null    float64
 1   sum             724 non-null    float64
 2   status          730 non-null    object 
 3   sale            724 non-null    object 
 4   new/current     724 non-null    object 
 5   Unnamed: 5      0 non-null      float64
 6   document        709 non-null    object 
 7   receiving_date  607 non-null    object 
dtypes: float64(3), object(5)
memory usage: 45.8+ KB


Сначала необходимо привести данные в минимальный порядок. В частности, перенести наименование отчетного периода (месяц год) из столбца *status* в пустующую колонку *Unnamed: 5*. Но перед этим ее необходимо переименовать:

In [4]:
data = data.rename(columns={'Unnamed: 5': 'period'})

Выделить информацию о периоде из столбца *status* можно по столбцу *client_id*. Пропуски в нем содержатся только в нужных строках.

In [5]:
data.query('client_id.isnull()')

,client_id,sum,status,sale,new/current,period,document,receiving_date
0,NaN,NaN,Май 2021,NaN,NaN,NaN,NaN,NaN
129,NaN,NaN,Июнь 2021,NaN,NaN,NaN,NaN,NaN
258,NaN,NaN,Июль 2021,NaN,NaN,NaN,NaN,NaN
369,NaN,NaN,Август 2021,NaN,NaN,NaN,NaN,NaN
484,NaN,NaN,Сентябрь 2021,NaN,NaN,NaN,NaN,NaN
594,NaN,NaN,Октябрь 2021,NaN,NaN,NaN,NaN,NaN


In [6]:
# выделение информации о периоде из *status* в *period*
data.loc[0:129, 'period'] = data.loc[0:129, 'period'].fillna('Май 2021')
data.loc[129:258, 'period'] = data.loc[129:258, 'period'].fillna('Июнь 2021')
data.loc[258:369, 'period'] = data.loc[258:369, 'period'].fillna('Июль 2021')
data.loc[369:484, 'period'] = data.loc[369:484, 'period'].fillna('Август 2021')
data.loc[484:594, 'period'] = data.loc[484:594, 'period'].fillna('Сентябрь 2021')
data.loc[594:730, 'period'] = data.loc[594:730, 'period'].fillna('Октябрь 2021')

In [7]:
# удаление ненужных строк
data = data.dropna(subset=['client_id'])
data.head()

,client_id,sum,status,sale,new/current,period,document,receiving_date
1,6.0,11693.7,ВНУТРЕННИЙ,-,-,Май 2021,-,-
2,14.0,5452.7,ОПЛАЧЕНО,Петрова,новая,Май 2021,оригинал,2021-07-26 00:00:00
3,15.0,13991.7,ОПЛАЧЕНО,Иванов,текущая,Май 2021,оригинал,2021-06-08 00:00:00
4,16.0,719.7,ОПЛАЧЕНО,Иванов,текущая,Май 2021,оригинал,2021-06-08 00:00:00
5,18.0,4253.7,ОПЛАЧЕНО,Иванов,текущая,Май 2021,оригинал,2021-07-07 00:00:00


Следующий этап предобработки - поиск и замена пропусков. 

In [8]:
data.isnull().sum()

client_id           0
sum                 0
status              0
sale                0
new/current         0
period              0
document           15
receiving_date    117
dtype: int64

Пропущенные значения присутствуют в столбцах *document* и *receiving_date*. 

In [15]:
data.query('receiving_date.isnull() or document.isnull()').sample(5)

,client_id,sum,status,sale,new/current,period,document,receiving_date
48,90.0,16123.70,ОПЛАЧЕНО,Смирнов,текущая,Май 2021,NaN,2021-07-28 00:00:00
646,294.0,5133.70,ОПЛАЧЕНО,Соколов,текущая,Октябрь 2021,НЕТ,NaN
720,382.0,3118.41,НА ПОДПИСАНИИ,Кузнецова,новая,Октябрь 2021,НЕТ,NaN
636,357.0,3753.70,ОПЛАЧЕНО,Михайлов,текущая,Октябрь 2021,НЕТ,NaN
376,32.0,7101.70,ОПЛАЧЕНО,Филимонова,текущая,Август 2021,NaN,2021-10-06 00:00:00


In [18]:
# уникальные значения в столбце *document* при пропуске даты
data.query('receiving_date.isnull() and not document.isnull()')['document'].unique()

array(['НЕТ'], dtype=object)

In [19]:
# пропуск и там, и там
data.query('receiving_date.isnull() and document.isnull()')

,client_id,sum,status,sale,new/current,period,document,receiving_date
459,256.0,2351.2,ОПЛАЧЕНО,Филимонова,текущая,Август 2021,NaN,NaN
489,30.0,3833.7,В РАБОТЕ,Филимонова,текущая,Сентябрь 2021,NaN,NaN
689,228.0,6393.7,ОПЛАЧЕНО,Смирнов,текущая,Октябрь 2021,NaN,NaN
690,228.0,1603.7,ОПЛАЧЕНО,Смирнов,текущая,Октябрь 2021,NaN,NaN


Встречается три варианта: 
- пропуск в *receiving_date*, но есть значение 'НЕТ' в *document*
- пропуск в *document*, но есть значение в *receiving_date*
- пропуск и в том, и в другом столбце

При этом, в первом варианте пропуск выглядит логично, так как оригинал договора просто не предоставлен. Эти значения заполнять не надо. 

Во втором варианте указана дата получения оригинала договора, но не указано, предоставлен ли оригинал. В данном случае пропуски в *document* следует заменить на 'оригинал'. 

В третьем варианте не указано ни то, ни другое. Таких строки всего 4 и все они имеют "текущий" тип сделки. Заменить пропуски в данном случае логичным значением не получится.

In [21]:
data.loc[~data['receiving_date'].isnull(), 'document'] = (
    data.loc[~data['receiving_date'].isnull(), 'document'].fillna('оригинал')
)

Также следует заменить значения "-", также являющиеся пропусками, в первой строке на NaN.

In [22]:
data = data.replace('-', np.nan)
data.head()

,client_id,sum,status,sale,new/current,period,document,receiving_date
1,6.0,11693.7,ВНУТРЕННИЙ,NaN,NaN,Май 2021,NaN,NaT
2,14.0,5452.7,ОПЛАЧЕНО,Петрова,новая,Май 2021,оригинал,2021-07-26
3,15.0,13991.7,ОПЛАЧЕНО,Иванов,текущая,Май 2021,оригинал,2021-06-08
4,16.0,719.7,ОПЛАЧЕНО,Иванов,текущая,Май 2021,оригинал,2021-06-08
5,18.0,4253.7,ОПЛАЧЕНО,Иванов,текущая,Май 2021,оригинал,2021-07-07


Наконец, для упрощения проведения анализа следует привести столбцы *client_id* и *receiving_date* к ожидаемым типам данных:

In [24]:
data['client_id'] = data['client_id'].astype('int')
data['receiving_date'] = pd.to_datetime(data['receiving_date'])

In [25]:
data.head()

,client_id,sum,status,sale,new/current,period,document,receiving_date
1,6,11693.7,ВНУТРЕННИЙ,NaN,NaN,Май 2021,NaN,NaT
2,14,5452.7,ОПЛАЧЕНО,Петрова,новая,Май 2021,оригинал,2021-07-26
3,15,13991.7,ОПЛАЧЕНО,Иванов,текущая,Май 2021,оригинал,2021-06-08
4,16,719.7,ОПЛАЧЕНО,Иванов,текущая,Май 2021,оригинал,2021-06-08
5,18,4253.7,ОПЛАЧЕНО,Иванов,текущая,Май 2021,оригинал,2021-07-07


Теперь с данными можно работать.